In [1]:
import os
import pandas as pd
import json
from tqdm.auto import tqdm
tqdm.pandas()
from transformers import AutoModel, AutoTokenizer
import torch
from torch.utils.data import DataLoader
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from pyvi.ViTokenizer import tokenize
from transformers import AutoTokenizer, AdamW, get_linear_schedule_with_warmup
from transformers import DataCollatorWithPadding
from scipy.stats import pearsonr, spearmanr
import math
from sklearn.metrics import *

In [ ]:
AUTH_TOKEN = "insert_your_huggingface_token"

In [3]:
tokenizer = AutoTokenizer.from_pretrained('nguyenvulebinh/vi-mrc-base', use_auth_token=AUTH_TOKEN)
print(tokenizer.decode(tokenizer.encode("sinh viên đại học bách khoa hà nội")))

<s> sinh viên đại học bách khoa hà nội</s>


In [4]:
df = pd.read_csv("./processed/train_stage2_ranking.csv")

In [5]:
df

,question,answer,title,candidate,label,group
0,Đất nước nào không có quân đội,"Costa Rica, Iceland, Panama, Micronesia, Quần ...",Costa Rica,Costa Rica Costa Rica (Phiên âm: Cô-xta Ri-ca)...,0,0
1,Đất nước nào không có quân đội,"Costa Rica, Iceland, Panama, Micronesia, Quần ...",Quần đảo Marshall,"Quần đảo Marshall Quần đảo Marshall, tên chính...",0,0
2,Đất nước nào không có quân đội,"Costa Rica, Iceland, Panama, Micronesia, Quần ...","Tamarindo, Costa Rica","Tamarindo, Costa Rica Tamarindo là một thị xã ...",0,0
3,Đất nước nào không có quân đội,"Costa Rica, Iceland, Panama, Micronesia, Quần ...","Montezuma, Costa Rica","Montezuma, Costa Rica Montezuma là một thị xã ...",0,0
4,Đất nước nào không có quân đội,"Costa Rica, Iceland, Panama, Micronesia, Quần ...",Micronesia,"Micronesia Micronesia (, ), còn gọi là Tiểu Đả...",0,0
...,...,...,...,...,...,...
46075,trong thần thoại hy lạp vị thần tình yêu có tê...,Eros,Eros phaleratus,Eros phaleratus Eros phaleratus là một loài bọ...,0,4607
46076,trong thần thoại hy lạp vị thần tình yêu có tê...,Eros,Eros melanurus,Eros melanurus Eros melanurus là một loài bọ c...,0,4607
46077,trong thần thoại hy lạp vị thần tình yêu có tê...,Eros,Eros melanopterus,Eros melanopterus Eros melanopterus là một loà...,0,4607
46078,trong thần thoại hy lạp vị thần tình yêu có tê...,Eros,Eros humeralis,Eros humeralis Eros humeralis là một loài bọ c...,0,4607


In [6]:
import torch.nn as nn
from transformers import AutoModel, AutoConfig

class PairwiseModel(nn.Module):
    def __init__(self, model_name):
        super(PairwiseModel, self).__init__()
        self.model = AutoModel.from_pretrained(model_name,use_auth_token=AUTH_TOKEN)
        self.config = AutoConfig.from_pretrained(model_name, use_auth_token=AUTH_TOKEN)
        self.drop = nn.Dropout(p=0.2)
        self.fc = nn.Linear(768, 1)
        
    def forward(self, ids, masks):
        out = self.model(input_ids=ids,
                           attention_mask=masks,
                           output_hidden_states=False).last_hidden_state
        out = out[:,0]
        outputs = self.fc(out)
        return outputs


In [7]:
from torch.utils.data import Dataset
tqdm.pandas()

class SiameseDataset(Dataset):

    def __init__(self, df, tokenizer, max_length):
        self.df = df
        self.max_length = max_length
        self.tokenizer = tokenizer
        self.df["content1"] = self.df.apply(lambda row: row.question+f" {tokenizer.sep_token} "+row.answer,axis=1)
        self.df["content2"] = self.df.apply(lambda row: row.title+f" {tokenizer.sep_token} "+row.candidate,axis=1)
        self.content1 = tokenizer.batch_encode_plus(list(df.content1.apply(lambda x: x.replace("_"," ")).values), max_length=max_length, truncation=True)["input_ids"]
        self.content2 = tokenizer.batch_encode_plus(list(df.content2.apply(lambda x: x.replace("_"," ")).values), max_length=max_length, truncation=True)["input_ids"]
        self.targets = self.df.label
        
    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        return {
            'ids1': torch.tensor(self.content1[index], dtype=torch.long),
            'ids2': torch.tensor(self.content2[index][1:], dtype=torch.long),
            'target': torch.tensor(self.targets[index], dtype=torch.float)
        }
pad_token_id = tokenizer.pad_token_id
def collate_fn(batch):
    ids = [torch.cat([x["ids1"], x["ids2"]]) for x in batch]
    targets = [x["target"] for x in batch]
    max_len = np.max([len(x) for x in ids])
    masks = []
    for i in range(len(ids)):
        if len(ids[i]) < max_len:
            ids[i]= torch.cat((ids[i], torch.tensor([pad_token_id,]*(max_len - len(ids[i])),dtype=torch.long)))
        masks.append(ids[i] != pad_token_id)
    # print(tokenizer.decode(ids[0]))
    outputs = {
        "ids": torch.vstack(ids),
        "masks": torch.vstack(masks),
        "target": torch.vstack(targets).view(-1)
    }
    return outputs

In [8]:
from sklearn.model_selection import GroupKFold, KFold

In [9]:
def optimizer_scheduler(model, num_train_steps):
    param_optimizer = list(model.named_parameters())
    no_decay = ["bias", "LayerNorm.weight"]
    optimizer_parameters = [
            {
                "params": [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
                "weight_decay": 0.001,
            },
            {
                "params": [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
                "weight_decay": 0.0,
            },
        ]

    opt = AdamW(optimizer_parameters, lr=3e-5)
    sch = get_linear_schedule_with_warmup(
        opt,
        num_warmup_steps=int(0.05*num_train_steps),
        num_training_steps=num_train_steps,
        last_epoch=-1,
    )
    return opt, sch

In [10]:
# tokenizer.decode(tokenizer.encode("Sinh viên trường Đại học Bách Khoa Hà Nội" ))

In [11]:
from sklearn.model_selection import GroupKFold
kfold = GroupKFold(n_splits=5)

In [ ]:
loss_fn = nn.BCEWithLogitsLoss()
epochs = 5
accumulation_steps = 8
scaler = torch.cuda.amp.GradScaler()
error_ids = None
for fold, (train_index, test_index) in enumerate(kfold.split(df, df.label, df.group)):
    model = PairwiseModel('nguyenvulebinh/vi-mrc-base')
    model.cuda()
    train_df = df
    # train_df = df.iloc[train_index].reset_index(drop=True)
    val_df = df.iloc[test_index].reset_index(drop=True)
    
    train_dataset = SiameseDataset(train_df, tokenizer, 256)
    valid_dataset = SiameseDataset(val_df, tokenizer, 256)
    train_loader = DataLoader(train_dataset, batch_size=4, collate_fn=collate_fn,
                              num_workers=2, shuffle=True, pin_memory=True, drop_last=True)
    valid_loader = DataLoader(valid_dataset, batch_size=32, collate_fn=collate_fn,
                              num_workers=2, shuffle=False, pin_memory=True)
    
    num_train_steps = len(train_loader) * epochs // accumulation_steps
    optimizer, scheduler = optimizer_scheduler(model, num_train_steps)
    
    for epoch in tqdm(range(epochs)):
        model.train()
        bar = tqdm(enumerate(train_loader), total=len(train_loader), leave=False)
        for step, data in bar:
            ids = data["ids"].cuda()
            # for x in ids:
            #     print(tokenizer.decode(x))
            masks = data["masks"].cuda()
            target = data["target"].cuda()
            # with torch.cuda.amp.autocast():
            preds = model(ids, masks)
            # print(preds.view(-1))
            loss = loss_fn(preds.view(-1), target.view(-1))
            loss /= accumulation_steps
            loss.backward()
            if (step + 1) % accumulation_steps == 0:
                optimizer.step()
                # scaler.update()
                optimizer.zero_grad()
                scheduler.step()
            bar.set_postfix(loss=loss.item())
        model.eval()
        with torch.no_grad():
            bar = tqdm(enumerate(valid_loader), total=len(valid_loader), leave=False)
            targets = []
            all_preds = []
            for step, data in bar:
                ids = data["ids"].cuda()
                masks = data["masks"].cuda()
                target = data["target"].cuda()
                preds = torch.sigmoid(model(ids, masks))
                all_preds.extend(preds.cpu().view(-1).numpy())
                targets.extend(target.cpu().view(-1).numpy())
            all_preds = np.array(all_preds)
            targets = np.array(targets)
        total = 0
        val_df["preds"] = all_preds
        for group in val_df.group.unique():
            tmp = val_df[val_df.group == group]
            if np.argmax(tmp.label.values) == np.argmax(tmp.preds.values):
                total += 1
        print(total/len(val_df.group.unique()))
    break

In [ ]:
torch.save(model.state_dict(), f"./outputs/pairwise_stage2_seed0.bin")